# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv from WeatherPy
weather_data = "../../WeatherPy/output_data/weather_file.csv"
weather_df = pd.read_csv(weather_data)
weather_df

,City,Country,City ID,Latitude,Longitude,Temperature,Max Temp,Humidity,Wind,Clouds,Date
0,Puerto Ayora,EC,3652764,-0.74,-90.35,69.01,69.01,94,3.00,91,1602727975
1,Portland,AU,2152667,-38.33,141.60,58.23,58.23,91,8.88,100,1602727978
2,Aswān,EG,359792,24.09,32.91,79.79,79.79,27,5.35,0,1602727979
3,Tamworth,AU,2147497,-31.10,150.93,78.80,78.80,31,4.70,0,1602727821
4,Natal,BR,3394023,-5.79,-35.21,77.00,77.00,83,8.05,40,1602727864
...,...,...,...,...,...,...,...,...,...,...,...
495,Preobrazheniye,RU,2017658,42.90,133.91,44.67,44.67,52,7.23,0,1602728584
496,Hofn,IS,2630299,64.25,-15.21,45.21,45.21,77,8.48,93,1602728585
497,Bissau,GW,2374775,11.86,-15.60,79.21,79.21,83,4.88,97,1602728587
498,Dolores,AR,3435038,-36.31,-57.68,46.15,46.15,74,11.81,0,1602728588


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
location1 = weather_df[["Latitude", "Longitude"]].astype(float)
humid=weather_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(location1, weights=humid,dissipating=False, max_intensity=10,point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Locate perfect weather conditions
perfectweather=weather_df.loc[(weather_df['Max Temp']>=70)&(weather_df['Max Temp']<=80)&(weather_df['Wind']<10)&(weather_df['Clouds']==0)]
perfectweather=perfectweather.dropna()
perfectweather

,City,Country,City ID,Latitude,Longitude,Temperature,Max Temp,Humidity,Wind,Clouds,Date
2,Aswān,EG,359792,24.09,32.91,79.79,79.79,27,5.35,0,1602727979
3,Tamworth,AU,2147497,-31.10,150.93,78.80,78.80,31,4.70,0,1602727821
73,Flinders,AU,6255012,-34.58,150.86,75.63,79.00,61,5.99,0,1602728064
83,Carutapera,BR,3402648,-1.20,-46.02,77.76,77.76,83,7.49,0,1602728076
100,Batemans Bay,AU,2176639,-35.72,150.18,72.90,78.01,71,8.99,0,1602728048
110,Jalu,LY,86049,29.03,21.55,71.01,71.01,38,8.16,0,1602728112
133,Oodweyne,SO,53372,9.41,45.06,70.81,70.81,73,2.73,0,1602728143
145,Pokhara,NP,1282898,28.23,83.98,73.92,73.92,54,1.34,0,1602728061
238,Sayhāt,SA,102318,26.49,50.04,71.60,71.60,83,3.36,0,1602728279
250,Bukama,CD,217834,-9.20,25.85,71.51,71.51,86,3.06,0,1602728294


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Using perfect weather dataframe, retrieve closest hotel of city using latitude and longitude coordinates
hotel_df=perfectweather
hotel_df["Hotel Name"]=""
hotel_df
for index, row in hotel_df.iterrows():
    param={"keyword":"hotel","radius":5000,"type":"lodging","key":g_key}
    url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lat=row['Latitude']
    long=row['Longitude']
    param["location"]=f"{lat},{long}"
    htlq=requests.get(url,params=param).json()
    hotel_info=htlq["results"]
    
    try:
        hotel_df.loc[index,"Hotel Name"]=hotel_info[0]["name"]
    except (KeyError,IndexError):
            print('Hotel Not Found')
hotel_df=hotel_df.dropna()
hotel_df

Hotel Not Found
Hotel Not Found


,City,Country,City ID,Latitude,Longitude,Temperature,Max Temp,Humidity,Wind,Clouds,Date,Hotel Name
2,Aswān,EG,359792,24.09,32.91,79.79,79.79,27,5.35,0,1602727979,Hotel Sofitel Legend Old Cataract Aswan
3,Tamworth,AU,2147497,-31.10,150.93,78.80,78.80,31,4.70,0,1602727821,Powerhouse Hotel Tamworth by Rydges
73,Flinders,AU,6255012,-34.58,150.86,75.63,79.00,61,5.99,0,1602728064,Lakeview Hotel Motel
83,Carutapera,BR,3402648,-1.20,-46.02,77.76,77.76,83,7.49,0,1602728076,HOTEL R. SOARES
100,Batemans Bay,AU,2176639,-35.72,150.18,72.90,78.01,71,8.99,0,1602728048,Batemans Bay Hotel
110,Jalu,LY,86049,29.03,21.55,71.01,71.01,38,8.16,0,1602728112,Jalu Hotel
133,Oodweyne,SO,53372,9.41,45.06,70.81,70.81,73,2.73,0,1602728143,Al shukri hotel
145,Pokhara,NP,1282898,28.23,83.98,73.92,73.92,54,1.34,0,1602728061,Hotel Rani Mahal
238,Sayhāt,SA,102318,26.49,50.04,71.60,71.60,83,3.36,0,1602728279,golden garden al corniche hotel
250,Bukama,CD,217834,-9.20,25.85,71.51,71.51,86,3.06,0,1602728294,


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
# Display figure
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# References: I made use of the following to complete this assignment: code from activities, Google, classmates